In [1]:
#Passo1:Importar bibliotecas
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import time
import urllib.parse
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

# --- PASSO 2: Definir a URL e fazer a requisição inicial ---
url = "https://www.drogariaveracruz.com.br/medicamentos/"
url_base = url

# Header para evitar bloqueio
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Criar sessão para reaproveitar conexões
session = requests.Session()
session.headers.update(headers)

# --- Funções auxiliares ---

def achar_nome(div):
    tag_h2 = div.find('h2', class_='title')
    if tag_h2:
        tag_a = tag_h2.find('a')
        if tag_a:
            return tag_a.get_text(strip=True)
    return "Nome não encontrado"

def achar_preco(div):
    tag_preco = div.find('p', class_="unit-price p-0")
    if tag_preco:
        return tag_preco.get_text().strip()
    return None

def achar_precopix(div):
    tag_precopix = div.find('p', class_="sale-price-pix")
    if tag_precopix:
        strong_tag = tag_precopix.find('strong')
        if strong_tag:
            return strong_tag.get_text().strip()
    return None 

def achar_total_paginas(soup_inicial):
    container_template = soup_inicial.find('div', class_='page-template')
    if container_template:
        divs_candidatas = container_template.find_all('div', class_='text-center pt-3')
        for div in divs_candidatas:
            texto = div.get_text(strip=True)
            if "Página" in texto:
                partes = texto.split()
                ultima_palavra = partes[-1]
                if ultima_palavra.isdigit():
                    return int(ultima_palavra)
    return 1

def achar_link(produto_html, url_base):
    tag_h2 = produto_html.find('h2', class_='title')
    if tag_h2:
        tag_a = tag_h2.find('a')
        if tag_a and 'href' in tag_a.attrs:
            link_relativo = tag_a.get('href')
            return urllib.parse.urljoin(url_base, link_relativo)
    return "Não encontrado"

def achar_precodesconto(produto_html):
    tag_venda = produto_html.find('p', class_="sale-price p-0")
    if tag_venda:
        strong_tag = tag_venda.find('strong')
        if strong_tag:
            return strong_tag.get_text(strip=True)
    return None

def extrair_detalhes_do_json(soup_detalhes):
    brand = "Não encontrada"
    code = "Não encontrado"
    script_tag = soup_detalhes.find('script', type='application/ld+json')
    if script_tag:
        try:
            raw_json = script_tag.string.strip()
            raw_json = raw_json.replace("\n", "").replace("\r", "").replace("\t", "")
            dados_json = json.loads(raw_json)
            if isinstance(dados_json, list):
                for item in dados_json:
                    if isinstance(item, dict) and 'brand' in item:
                        dados_json = item
                        break
            if 'brand' in dados_json and isinstance(dados_json['brand'], dict):
                brand = dados_json['brand'].get('name', "Não encontrada")
            code = dados_json.get('gtin13', "Não encontrado")
        except Exception as e:
            print("  -> Aviso: Erro ao ler JSON-LD:", e)
    return brand, code

def limpar_preco(preco_str):
    if not preco_str: return None
    try:
        return float(preco_str.replace("R$", "").strip().replace('.', '').replace(',', '.'))
    except (ValueError, TypeError):
        return None

# --- NOVO: baixar página com retry ---
def baixar_url(url, tentativas=3):
    for i in range(tentativas):
        try:
            r = session.get(url, timeout=20)
            if r.status_code == 200:
                return r
        except Exception:
            time.sleep(random.uniform(0.5, 2))
    return None

# --- Função para processar um produto ---
def processar_produto(produto, url_base):
    nome = achar_nome(produto)
    preco = achar_preco(produto)
    precopix = achar_precopix(produto)
    precodesconto = achar_precodesconto(produto)
    link_produto = achar_link(produto, url_base)

    brand, code = None, None
    if link_produto != "Não encontrado":
        responseprodutos = baixar_url(link_produto)
        if responseprodutos:
            soup_produto = BeautifulSoup(responseprodutos.content, 'html.parser')
            brand, code = extrair_detalhes_do_json(soup_produto)

    unit_price = limpar_preco(preco)
    discount_price = limpar_preco(precodesconto)
    pix_price = limpar_preco(precopix)

    discount = round(unit_price - discount_price, 2) if unit_price and discount_price else None
    pix_discount = round(unit_price - pix_price, 2) if unit_price and pix_price else None

    return {
        'Marca': brand,
        'GTIN': code,
        'Desconto': discount,
        'Preco com desconto': discount_price,
        'link': link_produto,
        'Nome': nome,
        'Desconto com pix': pix_discount,
        'Preco pix': pix_price,
        'Preco unitario': unit_price
    }

# --- PRIMEIRA PÁGINA para descobrir total ---
response = baixar_url(url)
soup = BeautifulSoup(response.content, 'html.parser')
total_paginas = achar_total_paginas(soup)

lista_de_produtos = []

# --- LOOP PRINCIPAL ---
for pagina in range(1, total_paginas + 1):
    url_pagina = f'https://www.drogariaveracruz.com.br/medicamentos/?p={pagina}'
    response = baixar_url(url_pagina)
    if not response:
        continue

    soup = BeautifulSoup(response.content, 'html.parser')
    div_produtos = soup.find_all('div', class_='li')

    # processa os produtos em paralelo
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(processar_produto, produto, url_base) for produto in div_produtos]
        for future in as_completed(futures):
            resultado = future.result()
            if resultado:
                lista_de_produtos.append(resultado)
                print(f"Extraído: {resultado['Nome']}")

    time.sleep(1)  # pequena pausa entre páginas

# --- FINAL ---
print("\nExtração concluída! Gerando tabela...")
df = pd.DataFrame(lista_de_produtos)
display(df)

Extraído: Blephagel 40g
Extraído: Materna Nestlé 30 comprimidos revestidos
Extraído: Lactosil Flora 30 cápsulas
Extraído: Glifage XR 500mg 30 comprimidos de liberação prolongada
Extraído: Rivaroxabana 20mg EMS 30 comprimidos revestidos
Extraído: Slinda 4mg 3 blísteres com 28 comprimidos revestidos cada
Extraído: Renu Fresh Solução Multiuso 355ml + 120ml + Estojo para Lentes
Extraído: Niquitin 21mg 7 adesivos transdérmicos transparentes
Extraído: Probid 30 cápsulas
Extraído: Calcitran Triflex 30 comprimidos
Extraído: Aradois 50mg 30 comprimidos revestidos
Extraído: Salicetil Infantil 100mg 10 comprimidos
Extraído: Teste de Gravidez Confira 1un
Extraído: Opti-Free Puremoist 300ml + 120ml + Estojo de Lentes
Extraído: Ofolato 90 comprimidos
Extraído: Corus 50mg 30 comprimidos revestidos
Extraído: VagiSoft 30g + 10 Aplicadores Descartáveis
Extraído: Neosaldina Solução Oral 15ml
Extraído: Previgrip Glóbulos Weleda 20g
Extraído: Niquitin 7mg 7 adesivos transdérmicos transparentes
Extraído: Pr

,Marca,GTIN,Desconto,Preco com desconto,link,Nome,Desconto com pix,Preco pix,Preco unitario
0,União Química,3662042000249,14.29,115.61,https://www.drogariaveracruz.com.br/gel-para-h...,Blephagel 40g,20.07,109.83,129.90
1,Nestlé,7891045031257,6.83,69.07,https://www.drogariaveracruz.com.br/suplemento...,Materna Nestlé 30 comprimidos revestidos,10.28,65.62,75.90
2,Apsen,7896637032247,NaN,59.90,https://www.drogariaveracruz.com.br/lactosil-f...,Lactosil Flora 30 cápsulas,NaN,56.91,NaN
3,Merck,7891721201806,4.30,7.65,https://www.drogariaveracruz.com.br/glifage-xr...,Glifage XR 500mg 30 comprimidos de liberação p...,4.68,7.27,11.95
4,Ems,7896004780511,213.19,29.07,https://www.drogariaveracruz.com.br/rivaroxaba...,Rivaroxabana 20mg EMS 30 comprimidos revestidos,214.64,27.62,242.26
...,...,...,...,...,...,...,...,...,...
4271,Eurofarma,7891317487348,NaN,NaN,https://www.drogariaveracruz.com.br/versa-80mg...,"VERSA - 80mg, 2 seringas preenchidas com 0,8mL...",NaN,NaN,NaN
4272,Abbott,7896255766135,NaN,NaN,https://www.drogariaveracruz.com.br/venocur-fi...,"Venocur Fit 263,2mg 60 comprimidos revestidos ...",NaN,NaN,NaN
4273,Sigma Pharma,7894916201353,NaN,NaN,https://www.drogariaveracruz.com.br/daforin-gt...,DAFORIN GTS C/20 ML,NaN,NaN,NaN
4274,Medley,7896422531788,NaN,NaN,https://www.drogariaveracruz.com.br/aixa-2mg-0...,"Aixa 2mg + 0,03mg 21 comprimidos revestidos",NaN,NaN,NaN
